In [ ]:
import pandas as pd
import os
import requests
import time
from datetime import datetime, timedelta
import pandas as pd
import csv
from datetime import datetime, timedelta

df = pd.read_csv('/content/merged_dataset.csv')

if not all([AUTH_TOKEN, COMPANY_ID, USER_ID]):
    raise ValueError("Необходимо задать все переменные окружения в .env файле (AUTH_TOKEN, COMPANY_ID, USER_ID)")

HEADERS = {
    'Authorization': AUTH_TOKEN,
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'CompanyID': COMPANY_ID,
    'UserID': USER_ID,
    'Referer': 'https://wildbox.ru/dashboard/search-tops-analysis/formed',
    'Sec-Fetch-Dest': 'empty', 'Sec-Fetch-Mode': 'cors', 'Sec-Fetch-Site': 'same-origin',
    'Time-Zone': 'Europe/Moscow',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'
}


DATE_TO = datetime.now().strftime('%Y-%m-%d')
DATE_FROM = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')

## Functions for API

In [15]:
def get_brand_details(brand_id: int) -> dict:
    """Получает информацию по бренду."""
    url = "https://wildbox.ru/api/wb_dynamic/brands/"
    params = {'brand_ids': brand_id, 'date_from': DATE_FROM, 'date_to': DATE_TO, 'extra_fields': 'rating,reviews'}
    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=30)
        response.raise_for_status()
        results = response.json().get('results', [])
        return results[0] if results else {}
    except requests.exceptions.RequestException: return {}

In [16]:
def get_product_details(product_id: int) -> dict:
    """Получает детальную информацию по одному товару."""
    url = "https://wildbox.ru/api/wb_dynamic/products/"
    extra_fields = ('orders,proceeds,in_stock_percent,lost_proceeds,orders_dynamic,proceeds_dynamic,quantity,price,discount,weighted_price,rating,seller,brand,subject,images,promos,sales_speed,old_price')
    params = {'product_ids': product_id, 'date_from': DATE_FROM, 'date_to': DATE_TO, 'extra_fields': extra_fields}
    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=30)
        response.raise_for_status()
        results = response.json().get('results', [])
        return results[0] if results else {}
    except requests.exceptions.RequestException: return {}

In [19]:
def get_seller_details(brand_id: int) -> dict:
    """Получает информацию по селлеру."""
    url = "https://wildbox.ru/api/wb_dynamic/sellers/"
    params = {'seller_ids': brand_id, 'date_from': DATE_FROM, 'date_to': DATE_TO, 'extra_fields': 'rating,reviews, name'}
    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=30)
        response.raise_for_status()
        results = response.json().get('results', [])
        return results[0] if results else {}
    except requests.exceptions.RequestException: return {}

## Product

In [ ]:
product_ids = df['Артикул']

results = []

for pid in product_ids:
    try:
        product_data = get_product_details(int(pid))
        if product_data:
            results.append(product_data)
        else:
            print(f'Нет данных для ID: {pid}')
    except Exception as e:
        print(f'Ошибка при обработке ID {pid}: {e}')

all_products = []
for product_list in results:
    all_products.extend(product_list)
fields = [
    "id", "rating"
]

rows = []
for p in results:
    total_days = 0
    for promo in p.get('promos', []):
      start = datetime.strptime(promo['start_date'], '%Y-%m-%d')
      end = datetime.strptime(promo['end_date'], '%Y-%m-%d')
      duration = (end - start).days + 1  
      total_days += duration

    row = {
        "id": p.get("id"),
        "rating": p.get("rating"),
    }
    rows.append(row)

csv_filename = "products_info_only_rating.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    writer.writerows(rows)

## Seller

In [24]:
seller_ids = df['seller_id']

results = []
i = 0
for pid in seller_ids:
    i += 1
    try:
        seller_data = get_seller_details(int(pid))
        if seller_data:
            results.append(seller_data)
            print(f"Получена {i}-ая строка")
        else:
            print(f'Нет данных для ID: {pid}')
    except Exception as e:
        print(f'Ошибка при обработке ID {pid}: {e}')

all_seller = []
for seller_list in results:
    all_seller.extend(seller_list)
fields = [
    "id", "rating", "reviews", "name"
]

rows = []
for p in results:
    row = {
        "id": p.get("id"),
        "rating": p.get("rating"),
        "reviews": p.get("reviews"),
        "name": p.get("name")
    }
    rows.append(row)


csv_filename = "seller_rating.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    writer.writerows(rows)

Получена 1-ая строка
Получена 2-ая строка
Получена 3-ая строка
Получена 4-ая строка
Получена 5-ая строка
Получена 6-ая строка
Получена 7-ая строка
Получена 8-ая строка
Получена 9-ая строка
Получена 10-ая строка
Получена 11-ая строка
Получена 12-ая строка
Получена 13-ая строка
Получена 14-ая строка
Получена 15-ая строка
Получена 16-ая строка
Получена 17-ая строка
Получена 18-ая строка
Получена 19-ая строка
Получена 20-ая строка
Получена 21-ая строка
Получена 22-ая строка
Получена 23-ая строка
Получена 24-ая строка
Получена 25-ая строка
Получена 26-ая строка
Получена 27-ая строка
Получена 28-ая строка
Получена 29-ая строка
Получена 30-ая строка
Получена 31-ая строка
Получена 32-ая строка
Получена 33-ая строка
Получена 34-ая строка
Получена 35-ая строка
Получена 36-ая строка
Получена 37-ая строка
Получена 38-ая строка
Получена 39-ая строка
Получена 40-ая строка
Получена 41-ая строка
Получена 42-ая строка
Получена 43-ая строка
Получена 44-ая строка
Получена 45-ая строка
Получена 46-ая стро

## Brand

In [40]:
brand_ids = df['brand_id']

results = []

i = 0
for pid in brand_ids:
    i += 1
    try:
        brand_data = get_brand_details(int(pid))
        if brand_data:
            results.append(brand_data)
            print(f"Получена {i}-ая строка")
        else:
            print(f'Нет данных для ID: {pid}')
    except Exception as e:
        print(f'Ошибка при обработке ID {pid}: {e}')

all_brand = []
for brand_list in results:
    all_brand.extend(brand_list)
fields = [
    "id", "rating", "reviews", "name"
]

rows = []
for p in results:
    row = {
        "id": p.get("id"),
        "rating": p.get("rating"),
        "reviews": p.get("reviews"),
        "name": p.get("name")
    }
    rows.append(row)

csv_filename = "brand_rating.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    writer.writerows(rows)

Получена 1-ая строка
Получена 2-ая строка
Получена 3-ая строка
Получена 4-ая строка
Получена 5-ая строка
Получена 6-ая строка
Получена 7-ая строка
Получена 8-ая строка
Получена 9-ая строка
Получена 10-ая строка
Получена 11-ая строка
Получена 12-ая строка
Получена 13-ая строка
Получена 14-ая строка
Получена 15-ая строка
Получена 16-ая строка
Получена 17-ая строка
Получена 18-ая строка
Получена 19-ая строка
Нет данных для ID: 0
Получена 21-ая строка
Получена 22-ая строка
Получена 23-ая строка
Получена 24-ая строка
Получена 25-ая строка
Получена 26-ая строка
Получена 27-ая строка
Получена 28-ая строка
Получена 29-ая строка
Получена 30-ая строка
Получена 31-ая строка
Получена 32-ая строка
Получена 33-ая строка
Нет данных для ID: 0
Получена 35-ая строка
Получена 36-ая строка
Получена 37-ая строка
Получена 38-ая строка
Получена 39-ая строка
Получена 40-ая строка
Получена 41-ая строка
Получена 42-ая строка
Получена 43-ая строка
Получена 44-ая строка
Получена 45-ая строка
Получена 46-ая строка

## Мерж таблиц

In [ ]:
seller_df = pd.read_csv("/content/seller_rating.csv")
seller_df = seller_df.rename(columns={
    'name': 'Поставщик',
    'rating': 'seller_rating',
    'reviews': 'seller_reviews'
})

united_df = pd.read_csv("/content/united_R.csv")

merged_df = united_df.merge(
    seller_df[['Поставщик', 'seller_rating', 'seller_reviews']],
    on='Поставщик',
    how='left'
)

merged_df

,Средняя цена без СПП,Средний чек,Коэффициент демпинга,Доля дней в наличии,Общая скидка без СПП,Упущенные заказы,"Количество категорий, шт.",Название,Поставщик,Бренд,Предмет,Позиция в выдаче,Стоимость за 1000 показов,quantity,count_day_promos,count_images,rating,seller_rating,seller_reviews
0,2337.40,2277.65,2.56,100.000000,31.87,0,12.0,Мойка высокого давления аккумуляторная,ИП Любочкин А. Н.,POW,Мойки высокого давления,1,3327.0,278,30,5,4.7,4.725000,1564.0
1,2811.27,2638.97,6.13,100.000000,62.10,17,12.0,Мойка высокого давления для автомобиля аккумул...,WAPS,WAPS,Мойки высокого давления,2,3501.0,316,21,10,5.0,4.700000,7251513.0
2,2592.67,2537.70,2.12,100.000000,47.07,0,12.0,Мойка высокого давления аккумуляторная автомойка,ИМ Матвеев Артем Владимирович,HORDER,Мойки высокого давления,3,2621.0,592,28,13,4.5,4.740000,1072305.0
3,2592.67,2537.70,2.12,100.000000,47.07,0,12.0,Мойка высокого давления аккумуляторная автомойка,ИМ Матвеев Артем Владимирович,HORDER,Мойки высокого давления,3,2621.0,592,28,13,4.5,4.740000,1072305.0
4,4363.04,3376.57,22.61,73.333333,61.16,0,11.0,Мультифункциональная аккумуляторная мойка высо...,"ООО ""ХОМ ШОППИНГ РАША""",Мой мир,Мойки высокого давления,4,3557.0,11,4,10,4.8,4.718571,3800800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835,4582.03,4428.19,3.36,100.000000,47.70,0,7.0,Лабубу Labubu V3 случайный цвет наклейка Набор...,ИП Крымский Д. Д.,CACH,Брелоки,198,250.0,122,18,6,4.1,4.320000,9965.0
2836,655.40,0.00,0.00,33.333333,80.10,0,0.0,Labubu лабубу Lovely Fashion,ИП Озманян Н. А.,0,Мягкие игрушки,204,255.0,20,1,4,0.0,5.000000,10.0
2837,1818.87,1594.53,12.33,100.000000,19.80,0,2.0,Лабубу labubu игрушка брелок,ИП Байрамукова А. И.,0,Мягкие игрушки,248,0.0,53,23,4,4.5,3.362500,15180.0
2838,545.93,530.29,2.87,100.000000,50.33,0,3.0,Сумка шоппер на молнии с принтом лабубу,"ООО ""АРСЕНАЛ""",Posuta,Сумки хозяйственные,255,250.0,4908,29,7,4.9,4.748000,435486.0
